d ##### Usecase:Get crimes data by type
###### Solutions are given for following problem statements
###### Problem statement-1:Get crimes data per day
###### Problem statement-2:Get crimes data per month
###### Problem statement-3:Get crimes data per year
###### Problem statement-4:Get TOP3 crime types based on number of incidents in RESIDENCE
###### Problem statement-5:Get yearly crimes data from 2001 to 2020 with count
###### Problem statement-6:Get yearly crime data of location BANK
###### Problem statement-7:Get yearly crimes data  of DOMESTIC incidents for all years with location 
###### Problem statement-8:Get yearly crimes data of crimetype BURGLARY happened in residence, park and apartments
###### Problem statement-9:Get yearly crimes data of THEFT incidents between 2015 and 2017
###### Problem statement-10:Get yearly crimes data of NARCOTICS incidents happened in AIRPORTS

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum,round,avg,rank, col,to_date,from_unixtime
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql.functions import unix_timestamp, lit
spark = SparkSession.builder.master('local').appName('getCrimesDatabyType').enableHiveSupport().getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions",10)

In [3]:
crimesdatadf = spark.read.csv("/FileStore/tables/crimes_data.csv",header=True,inferSchema=True,sep=",").toDF('ID','CaseNumber','Date','Block','IUCR','CrimeType','Description',     'LocationDescription','Arrest','Domestic','Beat','District','Ward','CommunityArea','FBICode','XCoordinate','YCoordinate','Year','UpdatedOn','Latitude','Longitude','Location')
crimesdatadf.show(5)

+--------+----------+---------------+--------------------+----+-------------------+--------------------+-------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+---------------+--------+---------+--------+
 ID|CaseNumber| Date| Block|IUCR| CrimeType| Description|LocationDescription|Arrest|Domestic|Beat|District|Ward|CommunityArea|FBICode|XCoordinate|YCoordinate|Year| UpdatedOn|Latitude|Longitude|Location|
+--------+----------+---------------+--------------------+----+-------------------+--------------------+-------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+---------------+--------+---------+--------+
11034701| JA366925| 1/1/2001 11:00| 016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...| RESIDENCE| false| false| 412| 4| 8| 45| 11| null| null|2001| 8/5/2017 15:50| null| null| null|
11227287| JB147188| 10/8/2017 3:00| 092XX S RACINE AVE| 281|CRIM SEXUAL ASSAULT| NON-AGGRAVATED| RESIDENCE| false| false|2222| 22| 21| 73| 2| null| null|2017|2/11/2018 15:57| null| null| null|
11227583| JB147595|3/28/2017 14:00| 026XX W 79TH ST| 620| BURGLARY| UNLAWFUL ENTRY| OTHER| false| false| 835| 8| 18| 70| 5| null| null|2017|2/11/2018 15:57| null| null| null|
11227293| JB147230| 9/9/2017 20:17|060XX S EBERHART AVE| 810| THEFT| OVER $500| RESIDENCE| false| false| 313| 3| 20| 42| 6| null| null|2017|2/11/2018 15:57| null| null| null|
11227634| JB147599|8/26/2017 10:00| 001XX W RANDOLPH ST| 281|CRIM SEXUAL ASSAULT| NON-AGGRAVATED| HOTEL/MOTEL| false| false| 122| 1| 42| 32| 2| null| null|2017|2/11/2018 15:57| null| null| null|
+--------+----------+---------------+--------------------+----+-------------------+--------------------+-------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+---------------+--------+---------+--------+
only showing top 5 rows

In [4]:
crimesdatadf.printSchema()

root
-- ID: integer (nullable = true)
-- CaseNumber: string (nullable = true)
-- Date: string (nullable = true)
-- Block: string (nullable = true)
-- IUCR: string (nullable = true)
-- CrimeType: string (nullable = true)
-- Description: string (nullable = true)
-- LocationDescription: string (nullable = true)
-- Arrest: boolean (nullable = true)
-- Domestic: boolean (nullable = true)
-- Beat: integer (nullable = true)
-- District: integer (nullable = true)
-- Ward: integer (nullable = true)
-- CommunityArea: integer (nullable = true)
-- FBICode: string (nullable = true)
-- XCoordinate: string (nullable = true)
-- YCoordinate: string (nullable = true)
-- Year: integer (nullable = true)
-- UpdatedOn: string (nullable = true)
-- Latitude: string (nullable = true)
-- Longitude: string (nullable = true)
-- Location: string (nullable = true)

In [5]:
crimesdataMap =crimesdatadf.select('ID','CaseNumber',to_date(from_unixtime(unix_timestamp('Date','MM/dd/yyyy'))).alias("CrimeDate"),
                                   'CrimeType','Description','LocationDescription','Arrest','Domestic','Year')
crimesdataMap.show(5,False)

+--------+----------+----------+-------------------+-----------------------------------+-------------------+------+--------+----+
ID |CaseNumber|CrimeDate |CrimeType |Description |LocationDescription|Arrest|Domestic|Year|
+--------+----------+----------+-------------------+-----------------------------------+-------------------+------+--------+----+
11034701|JA366925 |2001-01-01|DECEPTIVE PRACTICE |FINANCIAL IDENTITY THEFT OVER $ 300|RESIDENCE |false |false |2001|
11227287|JB147188 |2017-10-08|CRIM SEXUAL ASSAULT|NON-AGGRAVATED |RESIDENCE |false |false |2017|
11227583|JB147595 |2017-03-28|BURGLARY |UNLAWFUL ENTRY |OTHER |false |false |2017|
11227293|JB147230 |2017-09-09|THEFT |OVER $500 |RESIDENCE |false |false |2017|
11227634|JB147599 |2017-08-26|CRIM SEXUAL ASSAULT|NON-AGGRAVATED |HOTEL/MOTEL |false |false |2017|
+--------+----------+----------+-------------------+-----------------------------------+-------------------+------+--------+----+
only showing top 5 rows

###### Problem statement-1:Get crimes data per day

In [7]:
CrimesDataPerday = crimesdataMap.groupBy('CrimeDate','CrimeType').agg(count('CrimeType').alias("NumberofCrimePerTypes")).sort('CrimeDate','CrimeType')
CrimesDataPerDay.coalesce(1).write.csv("/FileStore/tables/CrimesDataPerDay", compression="none", header ='true')
CrimesDataPerday.show(20,False)

+----------+--------------------------+---------------------+
crimedate |CrimeType |NumberofCrimePerTypes|
+----------+--------------------------+---------------------+
2001-01-01|DECEPTIVE PRACTICE |1 |
2001-01-01|OFFENSE INVOLVING CHILDREN|1 |
2001-01-03|MOTOR VEHICLE THEFT |1 |
2001-01-07|ASSAULT |1 |
2001-01-09|CRIMINAL DAMAGE |1 |
2001-01-10|THEFT |1 |
2001-01-13|ARSON |1 |
2001-01-13|NARCOTICS |1 |
2001-01-14|ASSAULT |1 |
2001-01-18|THEFT |1 |
2001-01-18|WEAPONS VIOLATION |1 |
2001-01-20|MOTOR VEHICLE THEFT |1 |
2001-01-20|NARCOTICS |1 |
2001-01-21|DECEPTIVE PRACTICE |1 |
2001-01-21|NARCOTICS |1 |
2001-01-22|MOTOR VEHICLE THEFT |1 |
2001-01-24|THEFT |1 |
2001-01-29|NARCOTICS |1 |
2001-01-29|THEFT |1 |
2001-01-31|BURGLARY |1 |
+----------+--------------------------+---------------------+
only showing top 20 rows

###### Problem statement-2:Get crimes data per month

In [9]:
crimesDataPerMonth = crimesdataMap.groupBy(substring('CrimeDate',0,7).alias("CrimeMonth"),'CrimeType').agg(count('CrimeType').alias("NumberofCrimePerTypes")).sort('CrimeMonth','CrimeType')
crimesDataPerMonth.coalesce(1).write.csv("/FileStore/tables/crimesDataPerMonth", compression="none", header ='true')
crimesDataPerMonth.show(20,False)

+----------+--------------------------+---------------------+
CrimeMonth|CrimeType |NumberofCrimePerTypes|
+----------+--------------------------+---------------------+
2001-01 |ARSON |1 |
2001-01 |ASSAULT |2 |
2001-01 |BURGLARY |1 |
2001-01 |CRIMINAL DAMAGE |1 |
2001-01 |DECEPTIVE PRACTICE |2 |
2001-01 |MOTOR VEHICLE THEFT |3 |
2001-01 |NARCOTICS |4 |
2001-01 |OFFENSE INVOLVING CHILDREN|1 |
2001-01 |THEFT |4 |
2001-01 |WEAPONS VIOLATION |1 |
2001-02 |ASSAULT |1 |
2001-02 |BATTERY |1 |
2001-02 |CRIMINAL DAMAGE |2 |
2001-02 |CRIMINAL TRESPASS |1 |
2001-02 |NARCOTICS |3 |
2001-02 |ROBBERY |4 |
2001-02 |THEFT |6 |
2001-03 |ASSAULT |1 |
2001-03 |BATTERY |3 |
2001-03 |CRIM SEXUAL ASSAULT |2 |
+----------+--------------------------+---------------------+
only showing top 20 rows

###### Problem statement-3:Get crimes data per year

In [11]:
crimesDataPerYear = crimesdataMap.groupBy(substring('CrimeDate',0,4).alias("CrimeYear"),'CrimeType').agg(count('CrimeType').alias("NumberofCrimePerTypes")).sort('CrimeYear','CrimeType')
crimesDataPerYear.coalesce(1).write.csv("/FileStore/tables/crimesDataPerYear", compression="none", header ='true')
crimesDataPerYear.show(20,False)

+---------+--------------------------+---------------------+
CrimeYear|CrimeType |NumberofCrimePerTypes|
+---------+--------------------------+---------------------+
2001 |ARSON |1 |
2001 |ASSAULT |19 |
2001 |BATTERY |26 |
2001 |BURGLARY |5 |
2001 |CRIM SEXUAL ASSAULT |2 |
2001 |CRIMINAL DAMAGE |22 |
2001 |CRIMINAL SEXUAL ASSAULT |1 |
2001 |CRIMINAL TRESPASS |6 |
2001 |DECEPTIVE PRACTICE |9 |
2001 |LIQUOR LAW VIOLATION |2 |
2001 |MOTOR VEHICLE THEFT |24 |
2001 |NARCOTICS |37 |
2001 |OFFENSE INVOLVING CHILDREN|1 |
2001 |OTHER OFFENSE |8 |
2001 |PROSTITUTION |3 |
2001 |PUBLIC PEACE VIOLATION |3 |
2001 |ROBBERY |18 |
2001 |SEX OFFENSE |1 |
2001 |THEFT |70 |
2001 |WEAPONS VIOLATION |6 |
+---------+--------------------------+---------------------+
only showing top 20 rows

###### Problem statement-4:Get TOP3 crime types based on number of incidents in RESIDENCE

In [13]:
top3crimetypes = crimesdataMap.groupBy('CrimeType','LocationDescription').agg(count('CrimeType').alias("NumberofIncidents")).\
                                                       filter(crimesdataMap.LocationDescription=="RESIDENCE").\
                                                       sort('NumberofIncidents',ascending=[0]).limit(3)
top3crimetypes.coalesce(1).write.csv("/FileStore/tables/top3crimetypes", compression="none", header ='true')
top3crimetypes.show(5,False)

+--------------------------+-------------------+-----------------+
CrimeType |LocationDescription|NumberofIncidents|
+--------------------------+-------------------+-----------------+
DECEPTIVE PRACTICE |RESIDENCE |456 |
THEFT |RESIDENCE |201 |
OFFENSE INVOLVING CHILDREN|RESIDENCE |152 |
+--------------------------+-------------------+-----------------+

###### Problem statement-5:Get yearly crimes data from 2001 to 2020 with count

In [15]:
NoofcrimesPerYear = crimesdataMap.groupBy(substring('CrimeDate',0,4).alias("CrimeYear")).agg(count('CrimeType').alias("NumberofCrimePeryear")).sort('CrimeYear')
NoofcrimesPerYear.coalesce(1).write.csv("/FileStore/tables/NoofcrimesPerYear", compression="none", header ='true')
NoofcrimesPerYear.show(25,False)

+---------+--------------------+
CrimeYear|NumberofCrimePeryear|
+---------+--------------------+
2001 |264 |
2002 |1364 |
2003 |379 |
2004 |207 |
2005 |333 |
2006 |94 |
2007 |44 |
2008 |108 |
2009 |49 |
2010 |55 |
2011 |55 |
2012 |90 |
2013 |122 |
2014 |241 |
2015 |359 |
2016 |182 |
2017 |673 |
2018 |335 |
2019 |45 |
+---------+--------------------+

###### Problem statement-6:Get yearly crime data of location BANK

In [17]:
crimesDataofbanks = crimesdataMap.groupBy(substring('CrimeDate',0,4).alias("CrimeYear"),'CrimeType','LocationDescription'). \
                                  agg(count('CrimeType').alias("NumberofCrimePeryear")). \
                                  filter(crimesdataMap.LocationDescription=="BANK"). \
                                  sort('CrimeYear','CrimeType')
crimesDataofbanks.coalesce(1).write.csv("/FileStore/tables/crimesDataofbanks", compression="none", header ='true')
crimesDataofbanks.show(30,False)

+---------+------------------+-------------------+--------------------+
CrimeYear|CrimeType |LocationDescription|NumberofCrimePeryear|
+---------+------------------+-------------------+--------------------+
2001 |DECEPTIVE PRACTICE|BANK |1 |
2001 |THEFT |BANK |1 |
2002 |DECEPTIVE PRACTICE|BANK |5 |
2002 |ROBBERY |BANK |1 |
2002 |THEFT |BANK |4 |
2003 |DECEPTIVE PRACTICE|BANK |3 |
2003 |NARCOTICS |BANK |1 |
2005 |DECEPTIVE PRACTICE|BANK |4 |
2006 |DECEPTIVE PRACTICE|BANK |2 |
2009 |DECEPTIVE PRACTICE|BANK |1 |
2011 |DECEPTIVE PRACTICE|BANK |2 |
2012 |DECEPTIVE PRACTICE|BANK |1 |
2013 |DECEPTIVE PRACTICE|BANK |4 |
2014 |DECEPTIVE PRACTICE|BANK |1 |
2015 |DECEPTIVE PRACTICE|BANK |6 |
2016 |DECEPTIVE PRACTICE|BANK |2 |
2017 |DECEPTIVE PRACTICE|BANK |6 |
2017 |THEFT |BANK |1 |
2018 |BATTERY |BANK |1 |
2018 |DECEPTIVE PRACTICE|BANK |5 |
+---------+------------------+-------------------+--------------------+

###### Problem statement-7:Get yearly crimes data  of DOMESTIC incidents for all years with location

In [19]:
crimesDataofdomestic = crimesdataMap.groupBy(substring('CrimeDate',0,4).alias("CrimeYear"),'CrimeType','LocationDescription','Domestic'). \
                                  agg(count('CrimeType').alias("NumberofCrimes")). \
                                  filter(crimesdataMap.Domestic=="TRUE"). \
                                  sort('CrimeYear','CrimeType')
crimesDataofdomestic.coalesce(1).write.csv("/FileStore/tables/crimesDataofdomestic", compression="none", header ='true')
crimesDataofdomestic.show(20,False)

+---------+-----------------------+------------------------------+--------+--------------+
CrimeYear|CrimeType |LocationDescription |Domestic|NumberofCrimes|
+---------+-----------------------+------------------------------+--------+--------------+
2001 |ASSAULT |STREET |true |1 |
2001 |BATTERY |PARKING LOT/GARAGE(NON.RESID.)|true |1 |
2001 |BATTERY |APARTMENT |true |2 |
2001 |BATTERY |STREET |true |1 |
2001 |CRIMINAL SEXUAL ASSAULT|RESIDENCE |true |1 |
2001 |OTHER OFFENSE |RESIDENCE |true |1 |
2001 |ROBBERY |STREET |true |1 |
2001 |THEFT |RESIDENCE |true |1 |
2002 |ARSON |APARTMENT |true |1 |
2002 |ASSAULT |APARTMENT |true |1 |
2002 |ASSAULT |ALLEY |true |1 |
2002 |ASSAULT |RESIDENCE |true |2 |
2002 |ASSAULT |STREET |true |5 |
2002 |BATTERY |COMMERCIAL / BUSINESS OFFICE |true |1 |
2002 |BATTERY |PARK PROPERTY |true |1 |
2002 |BATTERY |RESIDENCE |true |11 |
2002 |BATTERY |STREET |true |21 |
2002 |BATTERY |HIGHWAY/EXPRESSWAY |true |2 |
2002 |BATTERY |SIDEWALK |true |5 |
2002 |BATTERY |APARTMENT |true |4 |
+---------+-----------------------+------------------------------+--------+--------------+
only showing top 20 rows

###### Problem statement-8:Get yearly crimes data of crimetype BURGLARY happened in residence, park and apartments

In [21]:
crimesDataofburglary = crimesdataMap.groupBy(substring('CrimeDate',0,4).alias("CrimeYear"),'CrimeType','LocationDescription'). \
                                  agg(count('CrimeType').alias("NumberofCrimes")). \
                                  where((crimesdataMap.CrimeType == "BURGLARY" ) & \
                                       ((crimesdataMap.LocationDescription.contains("RESIDENCE"))| \
                                       (crimesdataMap.LocationDescription.contains("PARK"))| \
                                       (crimesdataMap.LocationDescription.contains("APARTMENT")))). \
                                  sort('CrimeYear','LocationDescription')
crimesDataofburglary.coalesce(1).write.csv("/FileStore/tables/crimesDataofburglary", compression="none", header ='true')
crimesDataofburglary.show(30,False)

+---------+---------+------------------------------+--------------+
CrimeYear|CrimeType|LocationDescription |NumberofCrimes|
+---------+---------+------------------------------+--------------+
2001 |BURGLARY |PARK PROPERTY |1 |
2001 |BURGLARY |RESIDENCE |2 |
2002 |BURGLARY |APARTMENT |4 |
2002 |BURGLARY |PARKING LOT/GARAGE(NON.RESID.)|1 |
2002 |BURGLARY |RESIDENCE |5 |
2002 |BURGLARY |RESIDENCE-GARAGE |2 |
2003 |BURGLARY |APARTMENT |2 |
2003 |BURGLARY |CHA APARTMENT |1 |
2003 |BURGLARY |RESIDENCE |4 |
2003 |BURGLARY |RESIDENCE-GARAGE |2 |
2004 |BURGLARY |APARTMENT |1 |
2005 |BURGLARY |RESIDENCE-GARAGE |1 |
2006 |BURGLARY |APARTMENT |1 |
2007 |BURGLARY |APARTMENT |1 |
2008 |BURGLARY |RESIDENCE-GARAGE |1 |
2014 |BURGLARY |RESIDENCE |1 |
2015 |BURGLARY |RESIDENCE |1 |
2017 |BURGLARY |APARTMENT |4 |
2017 |BURGLARY |CHA APARTMENT |1 |
2017 |BURGLARY |RESIDENCE |7 |
2017 |BURGLARY |RESIDENCE-GARAGE |2 |
2018 |BURGLARY |APARTMENT |2 |
+---------+---------+------------------------------+--------------+

###### Problem statement-9:Get yearly crimes data of THEFT incidents between 2015 and 2017

In [23]:
crimesDataoftheft = crimesdataMap.groupBy('Year','CrimeType','LocationDescription'). \
                                  agg(count('CrimeType').alias("NumberofCrimes")). \
                                  where((crimesdataMap.CrimeType == "THEFT" ) & \
                                       ((crimesdataMap.Year>=2015)|(crimesdataMap.Year<=2017))).\
                                  sort('Year')
crimesDataoftheft.coalesce(1).write.csv("/FileStore/tables/crimesDataoftheft", compression="none", header ='true')
crimesDataoftheft.show(50,False)

+----+---------+------------------------------+--------------+
Year|CrimeType|LocationDescription |NumberofCrimes|
+----+---------+------------------------------+--------------+
2001|THEFT |OTHER |9 |
2001|THEFT |BANK |1 |
2001|THEFT |PARKING LOT/GARAGE(NON.RESID.)|11 |
2001|THEFT |LAKEFRONT/WATERFRONT/RIVERBANK|1 |
2001|THEFT |SIDEWALK |2 |
2001|THEFT |GROCERY FOOD STORE |1 |
2001|THEFT |SMALL RETAIL STORE |1 |
2001|THEFT |RESIDENCE |2 |
2001|THEFT |CTA PLATFORM |1 |
2001|THEFT |WAREHOUSE |2 |
2001|THEFT |STREET |32 |
2001|THEFT |PARK PROPERTY |4 |
2001|THEFT |AIRPORT/AIRCRAFT |2 |
2001|THEFT |HOTEL/MOTEL |1 |
2002|THEFT |GAS STATION |3 |
2002|THEFT |VEHICLE-COMMERCIAL |1 |
2002|THEFT |GROCERY FOOD STORE |13 |
2002|THEFT |LAKEFRONT/WATERFRONT/RIVERBANK|1 |
2002|THEFT |SIDEWALK |7 |
2002|THEFT |CTA PLATFORM |2 |
2002|THEFT |DRUG STORE |6 |
2002|THEFT |OTHER |23 |
2002|THEFT |APARTMENT |3 |
2002|THEFT |WAREHOUSE |3 |
2002|THEFT |CHA APARTMENT |1 |
2002|THEFT |RESTAURANT |8 |
2002|THEFT |ATHLETIC CLUB |3 |
2002|THEFT |CTA TRAIN |2 |
2002|THEFT |CTA GARAGE / OTHER PROPERTY |1 |
2002|THEFT |TAVERN/LIQUOR STORE |3 |
2002|THEFT |PARK PROPERTY |4 |
2002|THEFT |RESIDENCE PORCH/HALLWAY |1 |
2002|THEFT |SMALL RETAIL STORE |13 |
2002|THEFT |BANK |4 |
2002|THEFT |FACTORY/MANUFACTURING BUILDING|2 |
2002|THEFT |BAR OR TAVERN |1 |
2002|THEFT |CURRENCY EXCHANGE |1 |
2002|THEFT |CAR WASH |1 |
2002|THEFT |CONSTRUCTION SITE |3 |
2002|THEFT |STREET |90 |
2002|THEFT |GOVERNMENT BUILDING/PROPERTY |1 |
2002|THEFT |VEHICLE NON-COMMERCIAL |4 |
2002|THEFT |CHA PARKING LOT/GROUNDS |1 |
2002|THEFT |COMMERCIAL / BUSINESS OFFICE |4 |
2002|THEFT |RESIDENCE |28 |
2002|THEFT |PARKING LOT/GARAGE(NON.RESID.)|27 |
2002|THEFT |DEPARTMENT STORE |14 |
2002|THEFT |HOTEL/MOTEL |1 |
2002|THEFT |HOSPITAL BUILDING/GROUNDS |3 |
2002|THEFT |TAXICAB |2 |
+----+---------+------------------------------+--------------+
only showing top 50 rows

###### Problem statement-10:Get monthly crimes data of NARCOTICS incidents happened in AIRPORTS

In [25]:
crimesDataofnarcotics = crimesdataMap.groupBy(substring('CrimeDate',0,7).alias("CrimeMonth"),'CrimeType','LocationDescription'). \
                                  agg(count('CrimeType').alias("NumberofCrimes")). \
                                  where((crimesdataMap.CrimeType == "NARCOTICS" ) & \
                                       (crimesdataMap.LocationDescription.contains("AIRPORT"))).\
                                  sort('CrimeMonth')
crimesDataofnarcotics.coalesce(1).write.csv("/FileStore/tables/crimesDataofnarcotics", compression="none", header ='true')
crimesDataofnarcotics.show(20,False)

+----------+---------+-------------------+--------------+
CrimeMonth|CrimeType|LocationDescription|NumberofCrimes|
+----------+---------+-------------------+--------------+
2002-03 |NARCOTICS|AIRPORT/AIRCRAFT |1 |
2014-12 |NARCOTICS|AIRPORT/AIRCRAFT |4 |
2015-07 |NARCOTICS|AIRPORT/AIRCRAFT |1 |
2015-10 |NARCOTICS|AIRPORT/AIRCRAFT |2 |
2017-09 |NARCOTICS|AIRPORT/AIRCRAFT |1 |
2017-10 |NARCOTICS|AIRPORT/AIRCRAFT |1 |
2017-11 |NARCOTICS|AIRPORT/AIRCRAFT |1 |
+----------+---------+-------------------+--------------+